In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9625,2024-10-14,Paraguai Lnb,20:30,Olimpia,Colonias Gold,1.44,2.52,151.5,1.83,1.83,-6.5,2.01,1.68,8KCN6Dyh,0.694444,0.396825,0.546448,0.546448,0.091270,177.360,116.439359,0.656514,2.4,1.341641,0.559017,98.31,2.340,1.856664,0.793446,37.0,163.636,48.357845,0.295521,1.8,1.643168,0.912871,214.84,2.170,0.756340,0.348544,0.0,87,82,1.13,2.62,112.690,178.528,0.385695,0.000000,0.126474,-0.50,-0.100,-4.400000,0.000000,0.0,0.000000,0.04,0.008,190.000000,0.000000,0.0,0.000000
9626,2024-10-14,Polônia Liga De Basquete,13:00,Szczecin,GTK Gliwice,1.13,4.77,165.5,1.83,1.83,-12.5,2.05,1.65,hf7N5cDt,0.884956,0.209644,0.546448,0.546448,0.094599,151.512,30.673050,0.202446,0.6,1.341641,2.236068,159.75,1.914,0.399912,0.208941,-29.0,220.656,57.205656,0.259253,1.2,1.643168,1.369306,185.42,2.532,0.673142,0.265854,-2.0,71,73,2.25,2.54,124.108,334.986,0.872498,0.000000,0.152888,0.95,0.190,0.684211,0.712930,0.7,-0.012930,3.22,0.644,5.854037,0.330728,0.4,0.069272
9627,2024-10-14,Turquia Superliga,13:00,Darussafaka,Manisa,1.46,2.64,169.5,1.83,1.87,-5.5,2.01,1.69,U3bxPM3M,0.684932,0.378788,0.546448,0.534759,0.063719,343.448,229.636368,0.668620,0.0,0.000000,NaN,274.77,4.696,2.848347,0.606547,-83.0,192.686,50.905472,0.264189,0.6,1.341641,2.236068,170.20,2.388,0.766531,0.320993,-33.0,71,74,3.87,2.30,180.576,347.020,0.407018,0.015289,0.122310,-3.99,-0.798,-0.576441,0.525035,0.2,-0.325035,0.12,0.024,68.333333,0.407321,0.5,0.092679
9628,2024-10-14,Turquia Tbl,12:00,OGM Ormanspor,Gaziantep,2.72,1.38,160.5,1.88,1.78,4.5,1.90,1.77,CxstRBGa,0.367647,0.724638,0.531915,0.561798,0.092285,150.588,29.490875,0.195838,0.6,1.341641,2.236068,128.10,2.174,0.276007,0.126958,-74.0,161.904,76.541818,0.472761,1.2,1.643168,1.369306,117.04,2.082,1.006663,0.483508,-17.0,61,77,2.10,1.52,162.290,201.082,0.462206,0.038640,0.050095,-1.32,-0.264,-6.515152,0.459851,0.5,0.040149,-1.80,-0.360,-1.055556,0.545677,0.4,-0.145677
9629,2024-10-14,Brasil Nbb,20:00,Corinthians Paulista,Pato,1.43,2.53,153.5,1.88,1.78,-6.5,1.98,1.70,WdzYquO8,0.699301,0.395257,0.531915,0.561798,0.094558,170.726,53.397281,0.312766,1.2,1.643168,1.369306,131.95,2.264,1.321809,0.583838,1.0,172.718,50.559619,0.292729,1.2,1.643168,1.369306,140.70,2.468,0.915325,0.370877,1.0,65,67,2.03,2.10,153.182,184.992,0.392837,0.038640,0.107603,-0.60,-0.120,-3.583333,0.609735,0.5,-0.109735,-1.66,-0.332,-4.608434,0.410296,0.4,-0.010296
9630,2024-10-14,Brasil Nbb,19:30,Paulistano,Pinheiros,1.18,4.06,148.5,1.83,1.83,-11.5,1.95,1.72,hEVhMyW1,0.847458,0.246305,0.546448,0.546448,0.093763,153.358,48.280666,0.314823,0.6,1.341641,2.236068,89.78,2.186,0.897680,0.410650,-11.0,149.672,55.131264,0.368347,1.2,1.643168,1.369306,114.24,2.208,0.682437,0.309075,-37.0,67,68,1.34,1.68,118.158,225.340,0.777278,0.000000,0.088629,-1.64,-0.328,-0.548780,0.721770,0.5,-0.221770,-1.98,-0.396,-7.727273,0.352895,0.3,-0.052895
9631,2024-10-14,Brasil Nbb,20:00,Mogi,Botafogo,1.42,2.59,152.5,1.82,1.84,-6.5,1.96,1.71,xd25RW7s,0.704225,0.386100,0.549451,0.543478,0.090326,240.056,51.200950,0.213288,1.8,1.643168,0.912871,221.76,3.448,0.846475,0.245497,-14.0,408.724,228.131279,0.558155,1.2,1.643168,1.369306,337.77,5.756,4.350061,0.755744,-36.0,72,81,3.08,4.17,209.436,492.914,0.412626,0.007728,0.096336,5.51,1.102,0.381125,0.451341,0.7,0.248659,0.92,0.184,8.641304,0.2527

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
9644,23:30,Eua Nba,Los Angeles Clippers,Dallas Mavericks,1.69,Back Home
